Step1: imports

In [ ]:
import pandas as pd 
import numpy as np 

from src.cohort_utils import build_cohort_columns, cohort_retention



Step2: load data

In [81]:
df = pd.read_csv("../data/online_retail.csv", encoding="ISO-8859-1")
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


Step3: clean column names


In [82]:
df.columns = df.columns.str.lower()

Step4: handle dates 

In [83]:
df['invoicedate'] = pd.to_datetime(df['invoicedate'])
df['invoicedate'].min(), df['invoicedate'].max()

(Timestamp('2010-12-01 08:26:00'), Timestamp('2011-12-09 12:50:00'))

Step5: remove unusable rows

we can't do retention without customerid.

In [84]:
df = df.dropna(subset=['customerid'])


Also remove canceled orders 

In [85]:
df = df[~df['invoiceno'].astype(str).str.startswith('C')]

Remove negative or zero quantity:

In [86]:
df = df[df['quantity'] > 0]
df = df[df['unitprice'] > 0]

Step6: Create revenue

In [87]:
df['revenue'] = df['quantity'] * df['unitprice']

Step7: create order_month

In [88]:
df['order_month'] = df['invoicedate'].dt.to_period('M').dt.to_timestamp()



Step8: define cohort month (first purchase)

In [89]:
first_purchase = (
    df.groupby('customerid')['order_month']
      .min()
      .reset_index()
)

first_purchase.columns = ['customerid', 'cohort_month']

df = df.merge(first_purchase, on='customerid', how='left')

In [90]:
# FORCE datetime
df['order_month'] = pd.to_datetime(df['order_month'], errors='coerce')
df['cohort_month'] = pd.to_datetime(df['cohort_month'], errors='coerce')

Step9: cohort index (month since first purchase)

In [91]:
df['cohort_index'] = (
    (df['order_month'].dt.year - df['cohort_month'].dt.year) * 12 +
    (df['order_month'].dt.month - df['cohort_month'].dt.month) + 1
)

See distribution of cohort_index

In [92]:
df['cohort_index'].value_counts().sort_index().head(15)

cohort_index
1     118753
2      27865
3      26989
4      27254
5      25425
6      26961
7      23705
8      23604
9      23000
10     23229
11     20392
12     23453
13      7254
Name: count, dtype: int64

Force a viewof later months

In [93]:
df[df['cohort_index'] > 1][['customerid', 'order_month', 'cohort_month', 'cohort_index']].head(10)

,customerid,order_month,cohort_month,cohort_index
26174,18097.0,2011-01-01,2010-12-01,2
26175,18097.0,2011-01-01,2010-12-01,2
26176,18097.0,2011-01-01,2010-12-01,2
26177,18097.0,2011-01-01,2010-12-01,2
26178,18097.0,2011-01-01,2010-12-01,2
26179,18097.0,2011-01-01,2010-12-01,2
26180,18097.0,2011-01-01,2010-12-01,2
26181,18097.0,2011-01-01,2010-12-01,2
26182,18097.0,2011-01-01,2010-12-01,2
26183,16656.0,2011-01-01,2010-12-01,2


Check multiple months per customer

In [94]:
df.groupby('customerid')['order_month'].nunique().sort_values(ascending=False).head()

customerid
12971.0    13
13078.0    13
13069.0    13
17757.0    13
17735.0    13
Name: order_month, dtype: int64

Step 8: Final validation

In [95]:
print("\nAfter cleaning shape:", df.shape)
print(df[['customerid', 'order_month', 'cohort_month', 'cohort_index']].head(10))
print("\nUnique cohort_index values:", df['cohort_index'].nunique())



After cleaning shape: (397884, 12)
   customerid order_month cohort_month  cohort_index
0     17850.0  2010-12-01   2010-12-01             1
1     17850.0  2010-12-01   2010-12-01             1
2     17850.0  2010-12-01   2010-12-01             1
3     17850.0  2010-12-01   2010-12-01             1
4     17850.0  2010-12-01   2010-12-01             1
5     17850.0  2010-12-01   2010-12-01             1
6     17850.0  2010-12-01   2010-12-01             1
7     17850.0  2010-12-01   2010-12-01             1
8     17850.0  2010-12-01   2010-12-01             1
9     13047.0  2010-12-01   2010-12-01             1

Unique cohort_index values: 13


Step 9: Save cleaned dataset

In [96]:
df.to_csv("../data/cleaned_online_retail.csv", index=False)
